In [1]:
# Main import block
import numpy as np
import pandas as pd
import requests
import cfscrape 
from bs4 import BeautifulSoup
import datetime
from datetime import date, datetime

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import re
import asyncio


import time
from tqdm import tqdm

# Upload to google sheets
import gspread
#import df2gspread as d2g
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

# Upload to yandex
import yadisk

import signal
from contextlib import contextmanager

import warnings
warnings.filterwarnings('ignore')


In [2]:
##### Functions to maintain side manipulations #####


# Timeout class for reattempting connection
class TimeoutException(Exception):
    pass


@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException("Timed out!")

    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)


# Function to convert date from json
def date_format(date_raw):
    timestamp = date_raw / 1000
    date = datetime.fromtimestamp(timestamp)
    formatted_date = date.strftime("%Y-%m-%d %H:%M:%S")
    return formatted_date


def date_format_reverse():
    date_now = date.today()
    date_string = date_now.strftime("%Y-%m-%d %H:%M:%S")
    date_raw = datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S").timestamp()
    date_raw = date_raw * 1000
    return int(date_raw)


# Function for uploading dataframes into the google docs
def google_upload(df, sheet_name):
    # Params used to connect to google api
    scope = [
        "https://spreadsheets.google.com/feeds",
        "https://www.googleapis.com/auth/drive",
    ]
    credentials = ServiceAccountCredentials.from_json_keyfile_name(
        "./misc/macro-parser-lme-c2f2972b48fc.json", scope
    )  # security token
    gc = gspread.authorize(credentials)

    # Key params for connection to particular document
    spreadsheet_key = "1WhLiXRcdlkG7NCvHac9unC8ROt4lcbY7GxrOEdezZ9s"  # document id
    wks_name = sheet_name  # sheet name that we use
    df_array = df.to_numpy()
    df_new = pd.DataFrame(df_array, columns=df.columns)
    # df_new.reset_index(drop=True, inplace=True)
    d2g.upload(df_new, spreadsheet_key, wks_name, credentials=credentials)
    # print(f'Uploading to {sheet_name} completed')
    time.sleep(3)


# yandex upload
def yandex_upload():
    token = pd.read_csv("./misc/token.csv", header=None)
    token = token.iloc[0, 0]

    y = yadisk.YaDisk(token=token)

    try:
        y.upload("./data/LME_db.xlsx", "/macro_db/LME_db.xlsx")
    except:
        y.remove("/macro_db/LME_db.xlsx")
        y.upload("./data/LME_db.xlsx", "/macro_db/LME_db.xlsx")


# Session creation via proxy
def get_session(url):

    # Free proxy function
    def get_free_proxies():
        url = "https://free-proxy-list.net/"
        soup = BeautifulSoup(requests.get(url).content, "html.parser")

        raw_list = []
        proxies = dict()
        trs = soup.find("table").find_all("tr")  # main table

        for i in trs[1:]:
            raw_list.append(i.find_all("td"))  # list of raw data rows

        for i in range(len(raw_list)):  # creating working proxy list
            try:
                if raw_list[i][6].text == "yes":  # taking only https
                    proxies[raw_list[i][3].text] = (
                        f"{raw_list[i][0].text}:{raw_list[i][1].text}"
                    )
            except IndexError:
                continue

        adress = pd.Series(proxies)  # creating proxy series

        return adress

    # create session
    session = requests.Session()

    # random proxy
    proxy = get_free_proxies()
    counter = 0

    while counter <= len(proxy):
        try:
            with time_limit(7):
                random_proxy = proxy.sample().values[0]
                session.proxies = {"https": random_proxy}
                response = session.get(url)
                break

        except OSError:
            counter += 1
            pass

        except TimeoutException:
            # print("NBK_tenge timed out! Another attempt")
            counter += 1
            print(f"Attempt {counter+1}")

    return response

### This is the main function block ###

In [3]:
################################################################
##########################   LME ###############################
################################################################


def lme_selenium():
    def get_price(metall=""):
        url = f"https://www.lme.com/en/Metals/Non-ferrous/LME-{metall}#Summary"

        service = Service()
        driver = webdriver.Chrome(service=service)

        try:
            # driver.maximize_window()
            driver.get(url=url)

            time.sleep(2)

            driver.execute_script(
                "window.scrollTo(0, window.scrollY + window.innerHeight);"
            )
            driver.execute_script(
                "window.scrollTo(0, window.scrollY + window.innerHeight);"
            )

            time.sleep(2)

            html_code = driver.page_source

            soup = BeautifulSoup(html_code, "html.parser")
            data_raw = soup.find_all("td", class_="data-set-table__cell")
            price = pd.to_numeric(data_raw[1].text)
            return price

        except Exception as error:
            print(error)

        finally:
            driver.close()
            driver.quit()

    def get_date(metall="Aluminium"):
        url = f"https://www.lme.com/en/Metals/Non-ferrous/LME-{metall}#Summary"

        service = Service()
        driver = webdriver.Chrome(service=service)

        try:
            driver.maximize_window()
            driver.get(url=url)

            time.sleep(2)

            driver.execute_script(
                "window.scrollTo(0, window.scrollY + window.innerHeight);"
            )
            driver.execute_script(
                "window.scrollTo(0, window.scrollY + window.innerHeight);"
            )

            time.sleep(2)

            html_code = driver.page_source

            soup = BeautifulSoup(html_code, "html.parser")
            data_raw = soup.find_all("td", class_="data-set-table__cell")

            date_raw = soup.find_all("span", class_="data-set-tabs__content-detail")

            date_processed = date_raw[3].text.strip()

            date = pd.to_datetime(date_processed)

            return date

        except Exception as error:
            print(error)

        finally:
            driver.close()
            driver.quit()

    try:

        date = get_date("Aluminium")
        al_price = get_price("Aluminium")
        cu_price = get_price("Copper")
        zn_price = get_price("Zinc")
        nk_price = get_price("Nickel")
        ld_price = get_price("Lead")

        data_row = pd.DataFrame(
            {
                "date": date,
                "aluminium": al_price,
                "copper": cu_price,
                "lead": ld_price,
                "nickel": nk_price,
                "zink": zn_price,
            },
            index=[0],
        )

        historical_data = pd.read_excel("./data/LME_db.xlsx", index_col=0)

        historical_data = pd.concat([historical_data, data_row]).reset_index(drop=True)

        historical_data.drop_duplicates(inplace=True)

        with pd.ExcelWriter(
            "./data/LME_db.xlsx", date_format="YYYY-MM-DD", datetime_format="YYYY-MM-DD"
        ) as writer:
            historical_data.to_excel(writer, sheet_name="LME_non_ferrous")

        google_upload(historical_data, "LME_non_ferrous")

        return historical_data

    except TypeError as e:
        print(e)

In [4]:
################################################################
#########################   KITCO (old) ########################
################################################################


def kitco_db():
    url = "https://www.kitco.com/price/fixes/london-fix"

    scraper = cfscrape.create_scraper()
    scraped_data = scraper.get(url)

    soup = BeautifulSoup(scraped_data.text)

    temp_row = soup.find_all(
        "div",
        class_="grid grid-cols-5 border-b border-gray-200 text-center py-4 border-[rgba(0,0,0,0)]",
    )

    raw_row = str(temp_row).split("<")

    row = [
        raw_row[2].replace("div>", ""),
        raw_row[6].replace("!-- -->", ""),
        raw_row[8].replace("div>", ""),
        raw_row[12].replace("!-- -->", ""),
        raw_row[16].replace("!-- -->", ""),
    ]

    df_row = pd.DataFrame(row).T

    df_row.rename(
        columns={0: "Date", 1: "Gold", 2: "Silver", 3: "Platinum", 4: "Palladium"},
        inplace=True,
    )

    df_row["Date"] = pd.to_datetime(df_row["Date"])

    for column in df_row.columns.to_list()[1:]:
        df_row[column] = df_row[column].str.replace(",", "")
        df_row[column] = pd.to_numeric(df_row[column])

    db_frame = pd.read_excel("./data/kitko_db.xlsx", index_col=0, parse_dates=["Date"])

    final_df = pd.concat([db_frame, df_row])
    final_df.reset_index(drop=True, inplace=True)
    final_df.drop_duplicates(inplace=True)

    final_df.to_excel("../parser_beta/data/kitko_db.xlsx", sheet_name="kitco_metall")

    google_upload(final_df, "KITCO")

    return final_df

In [5]:
################################################################
#########################   KITCO (new) ########################
################################################################

def kitko_db_new():
    def kitco_raw():
        try:
            url = "https://www.kitco.com/price/fixes/london-fix"

            service = Service()
            driver = webdriver.Chrome(service=service)

            # driver.maximize_window()
            driver.get(url=url)

            time.sleep(2)

            driver.execute_script(
                "window.scrollTo(0, window.scrollY + window.innerHeight);"
            )
            driver.execute_script(
                "window.scrollTo(0, window.scrollY + window.innerHeight);"
            )

            time.sleep(2)

            html_code = driver.page_source

            return html_code

        except Exception as error:
            print(error)

        finally:
            driver.close()
            driver.quit()

    html_code = kitco_raw()

    soup = BeautifulSoup(html_code)

    response_row = soup.find_all(
        class_="grid grid-cols-5 border-b border-gray-200 text-center py-4 border-[rgba(0,0,0,0)] undefined bg-alt-row"
    )[0]
    
    
    response_row = str(response_row).split("<div>")
    response_row = response_row[1:]
    response_row = [i.replace("</div>", "") for i in response_row]


    date = response_row[0]
    date = pd.to_datetime(date)


    metalls_row = response_row[1:]
    metalls_row = [i.split("/")[-1] for i in metalls_row]
    metalls_row = [i.strip() for i in metalls_row]
    metalls_row = [i.replace(",", "") for i in metalls_row]
    metalls_row = [pd.to_numeric(i) for i in metalls_row]
    metalls_row.insert(0, date)
    
    
    data = pd.DataFrame(metalls_row).T
    data.rename(
        columns={0: "Date", 1: "Gold", 2: "Silver", 3: "Platinum", 4: "Palladium"},
        inplace=True,
    )
    data["Date"] = pd.to_datetime(data["Date"])
    
    db_frame = pd.read_excel('./data/kitko_db.xlsx', index_col=0, parse_dates=['Date'])

    final_df = pd.concat([db_frame, data])
    final_df.reset_index(drop=True, inplace=True)
    final_df.drop_duplicates(inplace=True)

    final_df.to_excel(
        '../parser_beta/data/kitko_db.xlsx', sheet_name='kitco_metall')
    
    google_upload(final_df, 'KITCO')

    return final_df


In [6]:
################################################################
#########################   LBMA (KITCO subs) ##################
################################################################


def lbma_prescious():
    url_gold = "https://prices.lbma.org.uk/json/gold_pm.json"
    url_silver = "https://prices.lbma.org.uk/json/silver.json"
    url_platinum = "https://prices.lbma.org.uk/json/platinum_pm.json"
    url_paladium = "https://prices.lbma.org.uk/json/palladium_pm.json"

    def get_raw_data(url, metall="Default"):
        scraper = cfscrape.create_scraper()
        scraped_data = scraper.get(url)

        # scraped_data = get_session(url=url)

        raw_data = pd.read_json(scraped_data.text)
        data = raw_data[["d", "v"]]
        data["v"] = data["v"].apply(lambda x: x[0])
        data["d"] = pd.to_datetime(data["d"])

        data.rename(columns={"d": "Date", "v": metall}, inplace=True)
        data = data.tail(4)

        return data

    gold = get_raw_data(url_gold, metall="Gold")
    silver = get_raw_data(url_silver, metall="Silver")
    platinum = get_raw_data(url_platinum, metall="Platinum")
    paladium = get_raw_data(url_paladium, metall="Palladium")

    result_df = (
        gold.merge(silver, on="Date", how="outer")
        .merge(platinum, on="Date", how="outer")
        .merge(paladium, on="Date", how="outer")
    )

    result_df.fillna(value=0, inplace=True)
    result_df = result_df.sort_values(by="Date")
    result_df.reset_index(inplace=True, drop=True)

    historical = pd.read_excel("./data/lbme_kitco_subs.xlsx", index_col=0)

    result = pd.concat([historical, result_df], axis=0).reset_index(drop=True)

    result.drop_duplicates(inplace=True)

    result = result.sort_values(by="Date")

    result.to_excel("./data/lbme_kitco_subs.xlsx", sheet_name="lbme_metall")

    google_upload(result, "LBMA")

    return result

In [7]:
################################################################
####################   CB_currencies ###########################
################################################################


def cb_currencies():
    # current_year = datetime.datetime.now().year
    current_year = datetime.now().year

    dict_of_currencies = {
        "USD": "R01235",
        "EUR": "R01239",
        "Australian_Dollar": "R01010",
        "China_Yuan": "R01375",
        "British_Pound": "R01035",
        "Kazakhstan_Tenge": "R01335",
        "Japanese_Yen": "R01820",
        "Swiss_Franc": "R01775",
    }

    def get_data(currency):

        url = f'https://www.cbr.ru/currency_base/dynamics/?UniDbQuery.\
Posted=True&UniDbQuery.so=1&UniDbQuery.mode=1&UniDbQuery.date_req1=&UniDbQuery\
.date_req2=&UniDbQuery.VAL_NM_RQ={currency}&UniDbQuery.From={"01.01.2022"}&UniDbQuery\
.To=31.12.{current_year}'

        number_of_tries = 0

        while number_of_tries < 20:
            try:
                scraper = cfscrape.create_scraper()
                scraped_data = scraper.get(url)

                preprocesed_data = scraped_data.text.replace(",", ".")

                df = pd.read_html(preprocesed_data, header=1)[0]

                base_name_list = df.columns.to_list()
                rename_list = ["Date", "Nominal", "Value"]
                columns = dict(zip(base_name_list, rename_list))
                df.rename(columns=columns, inplace=True)
                df["Value"] = df["Value"] / df["Nominal"]
                df["Date"] = pd.to_datetime(df["Date"], dayfirst=True)
                df.sort_values(by="Date", inplace=True)
                df.reset_index(drop=True, inplace=True)

                return df
                break

            except:
                # print('Error occured!')
                time.sleep(1)
                number_of_tries += 1

    for key, value in dict_of_currencies.items():
        table = get_data(value)

        db_frame = pd.read_excel(f"./data/centrobank/{key}.xlsx", index_col=0)

        new_frame = pd.concat([table, db_frame]).drop_duplicates()

        print(key, new_frame.tail())

        google_upload(new_frame, f"{key}")

        with pd.ExcelWriter(
            f"./data/centrobank/{key}.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            new_frame.to_excel(writer, sheet_name=f"{key}")

In [8]:
################################################################
####################   CB_metalls ##############################
################################################################


def cb_metalls():
    current_year = datetime.now().year

    url = f"https://www.cbr.ru/hd_base/metall/metall_base_new/?UniDbQuery.Posted\
=True&UniDbQuery.From=01.01.2022&UniDbQuery.To=30.12.{current_year}&UniDbQuery.Gold\
=true&UniDbQuery.Silver=true&UniDbQuery.Platinum=true&UniDbQuery.Palladium\
=true&UniDbQuery.so=1"

    number_of_tries = 0

    while number_of_tries < 20:
        try:
            scraper = cfscrape.create_scraper()
            scraped_data = scraper.get(url)

            preprocesed_data = scraped_data.text.replace(",", ".")

            df = pd.read_html(preprocesed_data, header=0)[0]

            base_name_list = df.columns.to_list()
            rename_list = ["Date", "Gold", "Silver", "Platinum", "Palladium"]
            columns = dict(zip(base_name_list, rename_list))
            df.rename(columns=columns, inplace=True)

            for i in df.columns.to_list():
                try:
                    df[i] = df[i].str.replace(" ", "")
                except AttributeError:
                    pass

            df["Date"] = pd.to_datetime(df["Date"], dayfirst=True)
            df.sort_values(by="Date", inplace=True)
            df.reset_index(drop=True, inplace=True)

            df.sort_values(by="Date", inplace=True)

            db_frame = pd.read_excel("./data/centrobank/metalls.xlsx", index_col=0)

            result_df = pd.concat([df, db_frame]).drop_duplicates()

            google_upload(result_df, f"CB_Metalls_consolidate")

            with pd.ExcelWriter(
                f"./data/centrobank/metalls.xlsx",
                date_format="YYYY-MM-DD",
                datetime_format="YYYY-MM-DD",
            ) as writer:
                result_df.to_excel(writer, sheet_name=f"cb_metalls")

            print(result_df.tail())
            break

        except:
            # print('Error occured!')
            time.sleep(1)
            number_of_tries += 1

In [9]:
################################################################
#########################   NBK ################################
################################################################

def nbk_tenge():
    # Realy unrelieable source, mb it would be better off with using ms query inside the file
    try:
        year = date.today().year

        upper_bound = f'01.01.2022'
        lower_bound = f'31.12.{year}'

        url = f'https://nationalbank.kz/ru/exchangerates/ezhednevnye-oficialnye-rynochnye-kursy-valyut\
            /report?rates%5B%5D=5&beginDate={upper_bound}&endDate={lower_bound}'

        counter = 0

        while counter <= 6:
            try:
                with time_limit(15):
                    page = requests.get(url=url)
                    break

            except TimeoutException:
                # print("NBK_tenge timed out! Another attempt")
                counter += 1

        temp_df = pd.read_html(page.text)
        df = temp_df[0]
        df['Unnamed: 0'] = pd.to_datetime(df['Unnamed: 0'])  # , dayfirst=True)
        df.rename(columns={'Unnamed: 0': 'date'}, inplace=True)

        with pd.ExcelWriter(
                '../parser_beta/data/nbk_tenge.xlsx') as writer:
            df.to_excel(writer, sheet_name='tenge')

        # print('NBK_tenge parsing is DONE!')

        google_upload(df, 'nbk_tenge')

        return df
    
    except ValueError:
        print("Probably tech problem, check the source")

In [10]:
################################################################
##########################  SHMET ##############################
################################################################


def shmet_optimized():
    url = "https://en.shmet.com/api/rest/enweb/spot/getSpotPrice?code=baseMetal&size=10&currentLength=0"
    responce = requests.get(url)

    day_df = pd.DataFrame(responce.json()["data"])
    day_df["date"] = date.today()
    cooper_row = day_df[day_df["name"].str.contains("cu", case=False)]

    result = cooper_row[["date", "middle", "unit"]]
    result["date"] = pd.to_datetime(result["date"])
    result = result.rename(columns={"middle": "price"})

    hist_data = pd.read_excel("./data/shmet_historical.xlsx", index_col=0)

    new_df = pd.concat([result, hist_data], axis=0).reset_index(drop=True)
    new_df.drop_duplicates(inplace=True)

    with pd.ExcelWriter(
        "../parser_beta/data/shmet_historical.xlsx",
        date_format="YYYY-MM-DD",
        datetime_format="YYYY-MM-DD",
    ) as writer:
        new_df.to_excel(writer, sheet_name="SHMET")

    final = pd.read_excel("./data/shmet_historical.xlsx", index_col=0)
    final.drop_duplicates(inplace=True)

    with pd.ExcelWriter(
        "../parser_beta/data/shmet_historical.xlsx",
        date_format="YYYY-MM-DD",
        datetime_format="YYYY-MM-DD",
    ) as writer:
        final.to_excel(writer, sheet_name="SHMET")

    google_upload(final, "SHMET")

    return final

In [11]:
################################################################
################## NEW_WESTMETALL###############################
################################################################

def new_westmetall():

    def get_data(metall, col_name):
        url = f'https://www.westmetall.com/en/markdaten.php?action=table&field=LME_{metall}_cash'
        response = requests.get(url=url)

        df = pd.read_html(response.text)[0][:30]
        data = df.iloc[:, :2]

        data = data.query("date != 'date'")
        data['date'] = pd.to_datetime(data['date'])
        data.iloc[:, 1] = pd.to_numeric(data.iloc[:, 1])
        data.rename(columns={data.columns[1]: col_name}, inplace=True)

        return data

    al = get_data(metall='Al', col_name='aluminium')
    cu = get_data(metall='Cu', col_name='copper')
    pb = get_data(metall='Pb', col_name='lead')
    ni = get_data(metall='Ni', col_name='nickel')
    zn = get_data(metall='Zn', col_name='zink')
    tn = get_data(metall='Sn', col_name='tin')

    result = pd.merge(al, cu, on='date', how='left').merge(pb, on='date', how='left').merge(
        ni, on='date', how='left').merge(zn, on='date', how='left').merge(tn, on='date', how='left')

    old_data = pd.read_excel('./data/LME_westmetall_db.xlsx', index_col=0)

    final_data = pd.concat([old_data, result], axis=0)

    final_data.drop_duplicates(subset='date', inplace=True)

    final_data.sort_values(by='date', inplace=True)

    final_data.reset_index(inplace=True, drop=True)

    with pd.ExcelWriter(
        "../parser_beta/data/LME_westmetall_db.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD") as writer:
        final_data.to_excel(writer, sheet_name='LME_westmetall')

    google_upload(final_data, 'LME_westmetall')

    return final_data

In [12]:
new_westmetall()

,date,aluminium,copper,lead,nickel,zink,tin
0,2022-01-04,2815.5,9660.0,2327.0,20730,3602.0,39400
1,2022-01-05,2866.0,9778.0,2343.0,20900,3660.0,39700
2,2022-01-06,2912.5,9565.0,2291.0,20480,3590.0,39650
3,2022-01-07,2919.5,9615.0,2342.0,20725,3602.5,40650
4,2022-01-10,2923.0,9665.0,2303.0,21045,3576.5,40425
...,...,...,...,...,...,...,...
566,2024-04-03,2334.0,8932.0,2023.0,16870,2430.0,28025
567,2024-04-04,2394.5,9257.0,2084.5,17230,2551.5,28450
568,2024-04-05,2391.0,9198.0,2091.0,17475,2587.5,28705
569,2024-04-08,2417.0,9301.0,2111.0,17700,2627.0,29395


In [13]:
cb_currencies()

USD           Date  Nominal    Value
553 2024-04-04        1  92.3892
554 2024-04-05        1  92.3058
555 2024-04-06        1  92.4155
556 2024-04-09        1  92.5810
557 2024-04-10        1  92.7463
EUR           Date  Nominal     Value
553 2024-04-04        1   99.4277
554 2024-04-05        1  100.0668
555 2024-04-06        1  100.1259
556 2024-04-09        1  100.2236
557 2024-04-10        1  100.7473
Australian_Dollar           Date  Nominal    Value
553 2024-04-04        1  60.2285
554 2024-04-05        1  60.8018
555 2024-04-06        1  60.7355
556 2024-04-09        1  60.9090
557 2024-04-10        1  61.2868
China_Yuan           Date  Nominal    Value
553 2024-04-04        1  12.7145
554 2024-04-05        1  12.7143
555 2024-04-06        1  12.7602
556 2024-04-09        1  12.7696
557 2024-04-10        1  12.8004
British_Pound           Date  Nominal     Value
553 2024-04-04        1  116.1055
554 2024-04-05        1  116.4807
555 2024-04-06        1  117.0720
556 2024-04-09 

APIError: {'code': 429, 'message': "Quota exceeded for quota metric 'Read requests' and limit 'Read requests per minute per user' of service 'sheets.googleapis.com' for consumer 'project_number:913199229264'.", 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'RATE_LIMIT_EXCEEDED', 'domain': 'googleapis.com', 'metadata': {'quota_location': 'global', 'consumer': 'projects/913199229264', 'quota_limit': 'ReadRequestsPerMinutePerUser', 'service': 'sheets.googleapis.com', 'quota_metric': 'sheets.googleapis.com/read_requests', 'quota_limit_value': '60'}}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Request a higher quota limit.', 'url': 'https://cloud.google.com/docs/quota#requesting_higher_quota'}]}]}

In [ ]:
cb_metalls()

          Date     Gold  Silver Platinum Palladium
551 2024-04-02   6570.5   72.82  2691.28   3017.68
552 2024-04-03  6587.16   73.00  2698.11   3025.33
553 2024-04-04  6726.42   76.19   2747.6   3017.91
554 2024-04-05   6766.8   77.89  2756.99   3000.34
555 2024-04-06   6814.5   80.22   2778.1   3042.53


In [ ]:
#kitko_db_new()

In [14]:
lbma_prescious()

,Date,Gold,Silver,Platinum,Palladium
0,1968-01-02,0.00,2.173,0.0,0.0
1,1968-01-03,0.00,2.225,0.0,0.0
2,1968-01-04,0.00,2.171,0.0,0.0
3,1968-01-05,0.00,2.157,0.0,0.0
4,1968-01-08,0.00,2.163,0.0,0.0
...,...,...,...,...,...
14229,2024-04-04,2293.50,27.000,935.0,1024.0
14230,2024-04-05,2298.55,26.780,923.0,998.0
14231,2024-04-08,2320.25,27.825,0.0,0.0
14232,2024-04-08,2320.25,27.825,946.0,1037.0


In [15]:
nbk_tenge()

,date,Числовое значение,ДОЛЛАР США
0,2022-01-01,1,431.80
1,2022-01-02,1,431.80
2,2022-01-03,1,431.80
3,2022-01-04,1,431.80
4,2022-01-05,1,431.80
...,...,...,...
826,2024-04-06,1,446.78
827,2024-04-07,1,446.78
828,2024-04-08,1,446.78
829,2024-04-09,1,445.95


In [16]:
shmet_optimized()


,date,price,unit
0,2024-04-10,75890,Yuan/MT
1,2024-04-09,75890,Yuan/MT
2,2024-04-09,75400,Yuan/MT
3,2024-04-06,73400,Yuan/MT
4,2024-04-05,73400,Yuan/MT
...,...,...,...
855,2020-01-17,48930,Yuan/MT
856,2020-01-16,48950,Yuan/MT
857,2020-01-15,49060,Yuan/MT
858,2020-01-14,48990,Yuan/MT


In [17]:
lme_selenium()

list index out of range
list index out of range
list index out of range
list index out of range
list index out of range


,date,aluminium,copper,lead,nickel,zink
0,2022-01-04,2815.5,9660.0,2327.0,20730.0,3602.0
1,2022-01-05,2866.0,9778.0,2343.0,20900.0,3660.0
2,2022-01-06,2912.5,9565.0,2291.0,20480.0,3590.0
3,2022-01-07,2919.5,9615.0,2342.0,20725.0,3602.5
4,2022-01-10,2923.0,9665.0,2303.0,21045.0,3576.5
...,...,...,...,...,...,...
567,2024-04-03,2334.0,8932.0,2023.0,16870.0,2430.0
568,2024-04-04,2394.5,9257.0,2084.5,17230.0,2551.5
569,2024-04-05,2391.0,9198.0,2091.0,17475.0,2587.5
570,2024-04-08,NaN,NaN,NaN,NaN,NaN
